<a href="https://colab.research.google.com/github/user257814938/Hackathon_1/blob/main/projet/version_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Introduction**

Dans un contexte où la fidélisation des employés est essentielle, l’attrition représente un défi majeur pour les entreprises. Comprendre pourquoi certains employés quittent leur poste et quels facteurs influencent leur performance permet de mettre en place des stratégies RH efficaces.

À partir du jeu de données IBM HR Analytics – Employee Attrition & Performance, ce projet vise à identifier les variables clés (âge, poste, satisfaction, équilibre vie pro/perso, salaire, etc.) qui impactent le turnover et la performance des employés.

**Problématique**

Quels sont les facteurs les plus déterminants de l’attrition et de la performance des employés, et comment l’entreprise peut-elle agir pour améliorer la rétention et la satisfaction du personnel ?

**Import des librairies**

In [39]:
import pandas as pd              # manipulation de données
import numpy as np               # outils numériques de base
import matplotlib.pyplot as plt  # graphiques de base
import seaborn as sns            # visualisations statistiques

**Chargement du dataset depuis GitHub**

In [40]:
url = "https://raw.githubusercontent.com/user257814938/Hackathon_1/refs/heads/main/Dataset/raw/IBM%20HR%20Analytics%20Employee%20Attrition%20%26%20Performance.csv"  # URL raw du CSV
df = pd.read_csv(url)                                                                                                                                                # charge le CSV dans un DataFrame

**Aperçu avant nettoyage**

In [41]:
df.columns                   # affiche la liste de toutes les colonnes du DataFrame avant nettoyage

Index(['Age', 'Attrition', 'BusinessTravel', 'DailyRate', 'Department',
       'DistanceFromHome', 'Education', 'EducationField', 'EmployeeCount',
       'EmployeeNumber', 'EnvironmentSatisfaction', 'Gender', 'HourlyRate',
       'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction',
       'MaritalStatus', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked',
       'Over18', 'OverTime', 'PercentSalaryHike', 'PerformanceRating',
       'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager'],
      dtype='object')

In [42]:
df.info()                    # types de colonnes et valeurs non nulles avant nettoyage

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1470 non-null   int64 
 1   Attrition                 1470 non-null   object
 2   BusinessTravel            1470 non-null   object
 3   DailyRate                 1470 non-null   int64 
 4   Department                1470 non-null   object
 5   DistanceFromHome          1470 non-null   int64 
 6   Education                 1470 non-null   int64 
 7   EducationField            1470 non-null   object
 8   EmployeeCount             1470 non-null   int64 
 9   EmployeeNumber            1470 non-null   int64 
 10  EnvironmentSatisfaction   1470 non-null   int64 
 11  Gender                    1470 non-null   object
 12  HourlyRate                1470 non-null   int64 
 13  JobInvolvement            1470 non-null   int64 
 14  JobLevel                

In [43]:
df.head(5)                   # aperçu des 5 premières lignes avant nettoyage

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [44]:
print(df.shape)              # dimensions (lignes, colonnes) avant nettoyage

(1470, 35)


**Vérifier doublons & valeurs manquantes**

In [45]:
missing = df.isna().sum().sort_values(ascending=False)  # nombre de valeur manquante par colonne

In [46]:
display(missing[missing > 0])                           # affiche uniquement les colonnes avec des valeur manquante

,0


In [47]:
print(df.duplicated().sum())                            # nombre de lignes dupliquées

0


**Pré-nettoyage**

In [48]:
df.drop_duplicates(inplace=True)                                   # supprime les doublons exacts

In [49]:
obj_cols = df.select_dtypes(include="object").columns.tolist()     # liste des colonnes texte
for c in obj_cols:                                                 # boucle sur colonnes texte
    df[c] = df[c].astype(str).str.strip()                          # enlève espaces en début et fin

In [50]:
df[obj_cols] = df[obj_cols].apply(pd.to_numeric, errors="ignore")  # conversion des colonnes texte en numériques

/tmp/ipython-input-2465838021.py:1: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[obj_cols] = df[obj_cols].apply(pd.to_numeric, errors="ignore")  # conversion des colonnes texte en numériques


**Aperçu après nettoyage**

In [52]:
df.columns                   # affiche la liste de toutes les colonnes du DataFrame après nettoyage

Index(['Age', 'Attrition', 'BusinessTravel', 'DailyRate', 'Department',
       'DistanceFromHome', 'Education', 'EducationField', 'EmployeeCount',
       'EmployeeNumber', 'EnvironmentSatisfaction', 'Gender', 'HourlyRate',
       'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction',
       'MaritalStatus', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked',
       'Over18', 'OverTime', 'PercentSalaryHike', 'PerformanceRating',
       'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager', 'Attrition_Flag'],
      dtype='object')

In [53]:
df.info()                    # types de colonnes et valeurs non nulles après nettoyage

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 36 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1470 non-null   int64 
 1   Attrition                 1470 non-null   object
 2   BusinessTravel            1470 non-null   object
 3   DailyRate                 1470 non-null   int64 
 4   Department                1470 non-null   object
 5   DistanceFromHome          1470 non-null   int64 
 6   Education                 1470 non-null   int64 
 7   EducationField            1470 non-null   object
 8   EmployeeCount             1470 non-null   int64 
 9   EmployeeNumber            1470 non-null   int64 
 10  EnvironmentSatisfaction   1470 non-null   int64 
 11  Gender                    1470 non-null   object
 12  HourlyRate                1470 non-null   int64 
 13  JobInvolvement            1470 non-null   int64 
 14  JobLevel                

In [54]:
df.head()                   # aperçu des lignes après nettoyage

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition_Flag
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,80,0,8,0,1,6,4,0,5,1
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,80,1,10,3,3,10,7,1,7,0
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,80,0,7,3,3,0,0,0,0,1
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,80,0,8,3,3,8,7,3,0,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,80,1,6,3,3,2,2,2,2,0


In [55]:
print(df.shape)              # dimensions (lignes, colonnes) après nettoyage

(1470, 36)


**Sauvegarde du dataset sur Colab**

In [56]:
df.to_csv("IBM HR Analytics Employee Attrition & Performance - processed.csv", index=False)   # export CSV nettoyé

**Affiche la liste de tous les fichiers dans Colab**

In [57]:
import os                     # importe le module "os" (Operating System)
print(os.listdir())           # affiche la liste de tous les fichiers et dossiers du répertoire courant

['.config', 'IBM HR Analytics Employee Attrition & Performance - processed.csv', 'sample_data']


**Télécharger le dataset nettoyé**

In [58]:
from google.colab import files                                                        # module pour les transferts locaux
files.download("IBM HR Analytics Employee Attrition & Performance - processed.csv")   # télécharge le fichier sur ton ordinateur

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Chargement du dataset clean depuis GitHub**

In [60]:
clean_url = "https://raw.githubusercontent.com/user257814938/Hackathon_1/refs/heads/main/Dataset/processed/IBM%20HR%20Analytics%20Employee%20Attrition%20%26%20Performance%20-%20processed.csv"   # URL raw du CSV clean
df_clean = pd.read_csv(clean_url)                                                                                                                                                                 # charge la version clean depuis GitHub

**Feature Engineering**

In [62]:
# 1) Création d'une variable binaire pour la cible

if "Attrition_Flag" not in df_clean.columns:                                               # vérifie si la colonne existe déjà
    df_clean["Attrition_Flag"] = (df_clean["Attrition"].str.upper() == "YES").astype(int)  # 1 = Yes, 0 = No

# 2) Création de tranches d'âge

df_clean["AgeGroup"] = pd.cut(                                     # découpe les âges en catégories
    df_clean["Age"],                                               # colonne Age à découper
    bins=[17, 30, 40, 50, 60],                                     # intervalles de classes
    labels=["Jeune", "Milieu carrière", "Expérimenté", "Senior"]   # noms des groupes
)

# 3) Ramener toutes les valeurs de revenu sur une échelle de 0 à 1 pour faciliter les comparaisons

min_income = df_clean["MonthlyIncome"].min()     # récupère la plus petite valeur de revenu mensuel
max_income = df_clean["MonthlyIncome"].max()     # récupère la plus grande valeur de revenu mensuel

df_clean["IncomeNorm"] = (df_clean["MonthlyIncome"] - min_income) / (max_income - min_income)   # applique la formule de normalisation min-max : (valeur - min) / (max - min) et crée une nouvelle colonne normalisée

# 4) Encodage de la variable OverTime (Yes/No → 1/0)

if "OverTime" in df_clean.columns:
    df_clean["OverTime_Flag"] = df_clean["OverTime"].str.upper().map({"YES": 1, "NO": 0})  # conversion en binaire

# 5) S'assurer que les colonnes ajoutées pendant le feature engineering existent bien dans le DataFrame

print("Nouvelles colonnes ajoutées :")                                                                            # affiche un titre pour plus de clarté
print([col for col in ["Attrition_Flag", "AgeGroup", "IncomeNorm", "OverTime_Flag"] if col in df_clean.columns])  # parcourt la liste des colonnes attendues et affiche uniquement celles qui existent dans df_clean

Nouvelles colonnes ajoutées :
['Attrition_Flag', 'AgeGroup', 'IncomeNorm', 'OverTime_Flag']


**Analyse exploratoire des données (EDA)**

In [61]:
print("Répartition Attrition :")                    # titre
if "Attrition" in df_clean.columns:                 # vérifie la colonne
    print(df_clean["Attrition"].value_counts(dropna=False))  # compte brut
    print((df_clean["Attrition"].value_counts(normalize=True)*100).round(1))  # en %
elif "Attrition_Flag" in df_clean.columns:          # fallback si flag binaire
    print(df_clean["Attrition_Flag"].value_counts(dropna=False))
    print((df_clean["Attrition_Flag"].value_counts(normalize=True)*100).round(1))

(1470, 36)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 36 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1470 non-null   int64 
 1   Attrition                 1470 non-null   object
 2   BusinessTravel            1470 non-null   object
 3   DailyRate                 1470 non-null   int64 
 4   Department                1470 non-null   object
 5   DistanceFromHome          1470 non-null   int64 
 6   Education                 1470 non-null   int64 
 7   EducationField            1470 non-null   object
 8   EmployeeCount             1470 non-null   int64 
 9   EmployeeNumber            1470 non-null   int64 
 10  EnvironmentSatisfaction   1470 non-null   int64 
 11  Gender                    1470 non-null   object
 12  HourlyRate                1470 non-null   int64 
 13  JobInvolvement            1470 non-null   int64 
 14  JobLevel     